In [3]:
import text_utilities as tu
%load_ext autoreload
%autoreload 2

In [4]:
wordset = tu.from_path('../data/experiment/verbs')

verb_dict = tu.count_syllables(wordset)
verb_dict.keys()
for i in range(1,9): # von mächtigster menge absteigend morpheme analysieren
    print(i, len(verb_dict[i]))

1 5010
2 27261
3 30123
4 14810
5 4455
6 1005
7 193
8 32


Tokenizer-Vokabular anhand des Token-Vokabulars erstellen:

In [13]:
"""mapping 1010101 comparing words left and right"""
maps = []
target = "verstehen"
l = len(target)
cnt = 0
cc1, cc2, cc3 = 0,0,0

for k in verb_dict:
    for v in verb_dict[k]:
        pair = (v, target)
        case = tu.match_ends(v, target)
        shorter = min(pair, key=len)
        longer = max(pair, key=len)
        diff = len(longer) - len(shorter)


        if case.get("any"):
            cnt+=1
            if diff:
                if case.get("first"):
                    cc1+=1
                    m = tu.wordmap(longer=longer, shorter=shorter)
                    while len(m) < l:
                        m.append(0) # padding
                    maps.append(m)

                if case.get("last"):
                    m = []
                    cc2+=1
                    m = tu.wordmap(longer=longer, shorter=shorter, start=diff)
                    while len(m) < l:
                        m.insert(0, 0) # padding
                    maps.append(m)


            else:
                cc3+=1
                m = tu.wordmap(longer=pair[0], shorter=pair[1])
                maps.append(m)

print(cc1, cc2, cc3)

5036 9195 2755


In [15]:
# Summe aller gleichlangen maps untereinander, bildet nur Tendenz ab
#itertools recipes
new_maps = [x for x in maps if len(x)==len(target)]
sum_new_maps = [sum(x) for x in zip(*new_maps)]
sum_new_maps

[6323, 7117, 6514, 2775, 1722, 2499, 1106, 5997, 10854]

In [16]:
# konvertiere map in string, nur vorübergehend?
str_maps = []
for m in maps:
    mstr = ""
    for i in m:
        mstr += str(i)
    str_maps.append(mstr)
print(str_maps[0], len(str_maps))

111000000 16986


In [17]:
# neue maps ignorieren unterschiede im wortinneren
import regex as rex
recount_map = []
for cnt in str_maps:
    sr = rex.sub(pattern='([^^1][0*1*]+[^$1])', repl=lambda m: len(m.group(1))*"0",string=cnt)
    recount_map.append(sr)

# recount_map

In [21]:
# word map after
regexed_listed = [list(i) for i in recount_map]
regexed_inted = []
for i in regexed_listed:
    regexed_inted.append([int(j) for j in i])

new_maps = [x for x in regexed_inted if len(x)==len(target)]
sum_new_maps = [sum(x) for x in zip(*new_maps)]

print(sum_new_maps)

[6323, 5573, 5537, 1246, 244, 174, 402, 4578, 10854]


In [24]:
import collections as cl
n_maps = cl.Counter(recount_map).most_common(50)
n_maps[:50]

[('000000001', 6206),
 ('111000000', 4219),
 ('000000011', 4068),
 ('111100000', 954),
 ('100000000', 714),
 ('000000111', 278),
 ('111110000', 153),
 ('111000011', 81),
 ('000001111', 80),
 ('111111000', 54),
 ('110000000', 34),
 ('000111111', 31),
 ('111100001', 26),
 ('100000001', 26),
 ('111100011', 21),
 ('111000001', 17),
 ('100000011', 8),
 ('111111100', 3),
 ('111001111', 3),
 ('111000111', 2),
 ('110000001', 2),
 ('111111111', 1),
 ('111100111', 1),
 ('111110011', 1),
 ('100001111', 1),
 ('111111110', 1),
 ('100000111', 1)]

In [44]:
def map_subword(target: str, map:str) -> str:
    """Returns a subword from a target string and a map. Yet to implement maps with 1 on both ends."""
    if map.startswith("1"):
        return target[:map.count("1")]+"##"
    elif map.endswith("1"):
        return "##"+target[-map.count("1"):]
print(map_subword("verstehen", "000000011"))

##en
